# Challenges

How can I contribute to the community quickly? 
Try to answer questions on stackoverflow. May due to lack of experience, I found it is difficult to understand what is happening just based on description.

- Search similar queries to get idea of root cause and solutions
- Search relevant manual description (do this in next iteration)
- Explain potential causes with relevant documents
- Try reproduction of problem and solution ... this part would be difficult? (first out of scope)

In [1]:
%run ./global.ipynb

In [2]:
from groq import Groq
from pymilvus import MilvusClient
groq_client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)
milvus_client = MilvusClient(DBFILEPATH)

Created new connection using: ffbb73a0786c4fa2ad1d3538b7f02d2e


In [3]:
milvus_client.describe_collection(STO_CLNAME)

{'collection_name': 'stackoverflow',
 'auto_id': True,
 'num_shards': 0,
 'description': 'Schema for stackoverflow',
 'fields': [{'field_id': 100,
   'name': 'id',
   'description': '',
   'type': <DataType.INT64: 5>,
   'params': {},
   'auto_id': True,
   'is_primary': True},
  {'field_id': 101,
   'name': 'vector_title',
   'description': '',
   'type': <DataType.FLOAT_VECTOR: 101>,
   'params': {'dim': 768}},
  {'field_id': 102,
   'name': 'vector',
   'description': '',
   'type': <DataType.FLOAT_VECTOR: 101>,
   'params': {'dim': 768}},
  {'field_id': 103,
   'name': 'question_title',
   'description': '',
   'type': <DataType.VARCHAR: 21>,
   'params': {'max_length': 10000}},
  {'field_id': 104,
   'name': 'question',
   'description': '',
   'type': <DataType.VARCHAR: 21>,
   'params': {'max_length': 10000}},
  {'field_id': 105,
   'name': 'url',
   'description': '',
   'type': <DataType.VARCHAR: 21>,
   'params': {'max_length': 1000}},
  {'field_id': 106,
   'name': 'answers'

In [8]:
# QA system part:


SYSTEM_PROMPT = """
We'd like to find out potential root cause for the reported issues on Milvus on Stackoverflow.
Given following information provided as JSON (link and contents), please give me potential causes, 
with what is the reason of your guess with relevant information URL.
"""


def get_potential_causes(query, milvus_client, groq_client):
    sto_docs = search_sto(query, milvus_client)
    prompt = f"""
    {SYSTEM_PROMPT}
    User's query:{query}
    Relevant stackoverflow past quetions:{sto_docs}
    Your answer:
    """
    chat_completion = groq_client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-70b-8192",
    )
    return chat_completion.choices[0].message.content


def search_sto(query, milvus_client):
    query_vectors = embed_sto([truncate_text(query)])
    search_params = {
        "metric_type": "COSINE"
    }
    res = milvus_client.search(
        collection_name=STO_CLNAME,  # target collection
        data=query_vectors,  # query vectors
        limit=5,  # number of returned entities
        search_params=search_params,
        anns_field="vector",
        output_fields=["question_title", "question", "answers", "url"],  # specifies fields to be returned
    )
    return [elem["entity"] for elem in res[0]]

In [10]:
query = """
I am looking for an option to export milvus data in ver 2.4 (sort of backup), and then migrate it to another machine. According to the doc, there is such option for migrating from ver 1,x. Is it possible to perform milvus-migration export from ver 2.4?

When I tried it I got error message indicating that the migration tool is attempting to use metadata database connection arguments. It seems the tool requires a specific mode to handle Milvus 2.x without MySQL..

(venv) qa@Ubuntu2204-2f3833:~/MILVUS-MIGRATION/work$ ./milvus-migration export --config=./configs/export_m2m.yaml
[2024/07/24 13:57:06.174 +03:00] [ERROR] [cmd/export.go:46] ["fail to export meta collection, empty meta db connect args"] [stack="github.com/zilliztech/milvus-migration/cmd.init.func1\n\t/home/qa/MILVUS-MIGRATION/milvus-migration/cmd/export.go:46\ngithub.com/spf13/cobra.(*Command).execute\n\t/home/qa/go/pkg/mod/github.com/spf13/cobra@v1.8.0/command.go:987\ngithub.com/spf13/cobra.(*Command).ExecuteC\n\t/home/qa/go/pkg/mod/github.com/spf13/cobra@v1.8.0/command.go:1115\ngithub.com/spf13/cobra.(*Command).Execute\n\t/home/qa/go/pkg/mod/github.com/spf13/cobra@v1.8.0/command.go:1039\ngithub.com/zilliztech/milvus-migration/cmd.Execute\n\t/home/qa/MILVUS-MIGRATION/milvus-migration/cmd/root.go:26\nmain.main\n\t/home/qa/MILVUS-MIGRATION/milvus-migration/main.go:16\nruntime.main\n\t/snap/go/10660/src/runtime/proc.go:271"]

(venv) qa@Ubuntu2204-2f3833:~/MILVUS-MIGRATION/work$ cat ./configs/export_m2m.yaml
dumper:
worker:
limit: 2
workMode: milvus1x
reader:
bufferSize: 1024
writer:
bufferSize: 1024

loader:
worker:
limit: 16

meta:
mode: none

source:
mode: local
local:
tablesDir: /home/qa/MILVUS/milvus/tables/

target:
mode: local
local:
outputDir: /home/qa/MILVUS/milvus/migration/output/
milvus2x:
endpoint: "localhost:19530"
username: xxxx
password: xxxx
"""


print(get_potential_causes(query, milvus_client, groq_client))

text was truncated: 701 -> 300


701


HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Retrying request to /openai/v1/chat/completions in 7.000000 seconds
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Based on the provided information, I'll try to identify potential causes for the reported issues on Milvus:

**Issue 1: Exporting Milvus data in version 2.4**

The user is trying to export Milvus data using the `milvus-migration` tool, but encounters an error indicating that the migration tool is attempting to use metadata database connection arguments.

Potential causes:

* The `milvus-migration` tool is not compatible with Milvus version 2.4.
* The configuration file `export_m2m.yaml` is not properly set up for Milvus version 2.4.

**Relevant information URL:** 
https://milvus.io/docs/v2.0.0/migration/export_data.md (Milvus documentation on exporting data)

**Issue 2: Pod has unbound immediate PersistentVolumeClaims**

The user is trying to deploy a Milvus cluster using Kubernetes, but encounters an error related to PersistentVolumeClaims.

Potential causes:

* The user has not configured Persistent Volumes (PVs) and StorageClasses correctly.
* The Milvus Helm chart is not properly c